<a href="https://colab.research.google.com/github/vu-topics-in-big-data-2022/Project-Incident-Team2/blob/master/plot_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install geopandas pandas pyarrow
import geopandas as gpd

     |████████████████████████████████| 1.0 MB 7.8 MB/s 
     |████████████████████████████████| 16.7 MB 192 kB/s 
     |████████████████████████████████| 6.3 MB 37.1 MB/s 


In [ ]:
roads=gpd.read_file('USA_Tennessee.geojson')
roads.head(2)